#### Course: Machine Learning 
#### Project: 01
#### Team members
#### + Mauro Travieso Pena
#### + Quoc Huy Luong
#### + Ngoc Bao Tran



# Overview of project


<div class="alert alert-block alert-info">
Some comments here 
</div>



# Import Libraries 

In [ ]:
import pandas as pd
import numpy as np
import random as rnd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import import_ipynb 
import functions as myLib




# Load data from files

In [ ]:
RawData = pd.read_csv('../data/anime.csv')

<div class="alert alert-block alert-info">
Some comments here 
</div>

# Data Descriptive 

#### View sample data

In [ ]:
data_columns = RawData.columns
data_columns
RawData.sample(5)


#### Check null values

In [ ]:
RawData.isnull().sum()


#### Drop null rows

In [ ]:
RawData = RawData.dropna()


#### remove unknown data

In [ ]:
RawData.info()

In [ ]:
RawData['episodes'].describe()

In [ ]:
RawData['episodes'].unique()


<div class="alert alert-block alert-danger">
We saw that, there are some 'Unknow' values in the <b>episodes</b> feature.
Should we remove these rows from the dataset ?
Let's check.
</div>

#### Check and remove Unknown data from episodes


In [ ]:
episodes_unknown = RawData[['anime_id','episodes','rating']].copy() 
episodes_unknown.sample(5)


In [ ]:
episodes_unknown.loc[episodes_unknown['episodes'] !='Unknown', 'episodes'] = 1
episodes_unknown.loc[episodes_unknown['episodes'] =='Unknown', 'episodes'] = 0
episodes_unknown.head()


In [ ]:
plt.scatter(episodes_unknown['episodes'],episodes_unknown['rating'] , alpha=0.5)

In [ ]:
episodes_unknown['episodes'].describe()


<div class="alert alert-block alert-danger">
Based on the outputs of the <b>episodes</b> feature,
we saw that the unknow values (0) does not affect the range of the rating.
So we can remove known values in the <b>episodes</b> feature.  
</div>



In [ ]:
RawData =  RawData[RawData['episodes']!='Unknown']
RawData.head()


In [ ]:
plt.scatter(RawData['episodes'],RawData['rating'] , alpha=0.5)



In [ ]:
#Check number of values in the type feature [6 uniques type]
#So if we do one-hot-encoder, we will have 6 columns
RawData['type'].describe()

In [ ]:
RawData.rating.plot(kind="kde")
plt.title("Rating Distribution")

In [ ]:
RawData['rating'].describe()

In [ ]:
ax = sns.boxplot(x="rating", 
                data=RawData)


In [ ]:
#Add a new column rating class 
RawData['Class']=1#RawData['rating']
RawData.sample(5)

In [ ]:
# 1: High
# or 0: Low based on rating
RawData.loc[RawData['rating'] >= RawData['rating'].mean(), 'Class'] = 1
RawData.loc[RawData['rating'] < RawData['rating'].mean(), 'Class'] = 0
RawData.sample(5)

In [ ]:
RawData['Class'].describe()

In [ ]:
RawData['Class'].hist(bins=3,figsize=(9,7),grid=False)


In [ ]:
#Split genre values into rows
NewData = pd.DataFrame(RawData.genre.str.split(',').tolist(), index=[RawData.anime_id,RawData.type,RawData.episodes,RawData.rating,RawData.members,RawData.Class]).stack()
NewData.head()

In [ ]:
NewData = NewData.reset_index([0,'anime_id','type','episodes','rating','members','Class'])
NewData.columns=['anime_id','type','episodes','rating','members','Class','genre']
NewData.head()

In [ ]:
#Check number of values in the genre feature [82 uniques type]
#So if we do one-hot-encoder, we will have 82 columns
NewData['genre'].describe()

In [ ]:
cor_matrix = NewData.corr().round(2)
fig = plt.figure(figsize=(12,12));
sns.heatmap(cor_matrix, annot=True, center=0, cmap = sns.diverging_palette(250, 10, as_cmap=True), ax=plt.subplot(111));
plt.show()

# IV. Visualization

In [ ]:
NewData.rating.plot(kind="kde")
plt.title("Rating Distribution")

In [ ]:
NewData['rating'].describe()

In [ ]:
ax = sns.boxplot(x="rating", 
                data=NewData)


#### Encoding Categorical Data


In [ ]:
#Encode type feature: 6 unique values
NewData_Encode = NewData.copy()
NewData_Encode = pd.get_dummies(NewData_Encode, columns=['type'], prefix = ['type'])

NewData_Encode.head()


In [ ]:
#Encode genre feature: 82 unique values
NewData_Encode = NewData_Encode.copy()
NewData_Encode = pd.get_dummies(NewData_Encode, columns=['genre'], prefix = ['genre'])

NewData_Encode.head()
NewData_Encode = NewData_Encode.drop(['rating'],axis=1)
NewData_Encode.head()



In [ ]:
#Drop anmie_id
Y_Data = NewData_Encode['Class']
NewData_Encode = NewData_Encode.drop(columns=['anime_id'])

NewData_Encode = NewData_Encode.drop(columns=['Class'])
NewData_Encode.head()

In [ ]:
Y_Data.head()



#### Normalize  Data


In [ ]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

Normalized_Data = min_max_scaler.fit_transform(NewData_Encode)
Normalized_Data
Normalized_Data = pd.DataFrame(Normalized_Data)
Normalized_Data.head()


#### PCA


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
pca.fit(Normalized_Data)
df = pca.transform(Normalized_Data)
PCA_Data = pd.DataFrame(df)
PCA_Data.head()
